In [1]:
import pandas as pd
import numpy as np
from lxml import etree
import xml.etree.ElementTree as ET
import time
import math

In [2]:
xml_positions = etree.parse('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##arbeitsplatop

In [3]:
kpimerged = pd.read_excel('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') #arbeit

In [4]:
path='C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/'    #arbeit

In [1]:
path='C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/'    #Privat

In [6]:
xml_positions = etree.parse('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##privat
kpimerged = pd.read_excel('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') ##privat

In [5]:
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)

In [6]:
class DataReader:
  """
  Main class to read in xml files provided by the DFB.

  Returns:
      ElementTree root: The root of the xml document.

  """

  def __init__(self,xml_file):
    self.xml_root = self._load_data(xml_file)


  def _load_data(self,xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root

  def create_dataframe(self,df_cols=None,findall_string=".//Object",time_search=None):
    """
    Create a dataframe from the xml inputs

    Args:
        df_cols (list): Optional, the column names. If no names given, columns
          detected automatically
        findall_string (str): The xpath string to iterate trees (https://www.w3schools.com/xml/xml_xpath.asp)
        time_search (float): Will currently look at parent if it's an Event, it
          will compare the time_search to the EventTime. I.e. this is a means of getting
          all passes before the frame time.

    Returns:
        pd.dataframe
    """

    populate_column_names=False
    if df_cols is None:
        df_cols = []
        populate_column_names=True

    out_df = pd.DataFrame(columns = df_cols)

    parent_map = dict((c, p) for p in self.xml_root.getiterator() for c in p)
    for node in self.xml_root.findall(findall_string):
        if time_search is not None:
          if (parent_map[node].tag != "Event"): continue
          event_time = parent_map[node].attrib["EventTime"]
          ts = ciso8601.parse_datetime(event_time)
          # to get time in seconds:
          seconds=time.mktime(ts.timetuple())
          # If timestamp is larger than the search, exit
          if seconds > time_search: continue

        # If column names are not provided, get them
        if populate_column_names==True:
            for key,value in node.attrib.items():
                df_cols.append(key)
                populate_column_names=False

        res = []
        for c in df_cols:
            res.append(node.attrib.get(c))
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index=True)
    return out_df


In [7]:
def get_tracking_data(xml_positions, half='firstHalf'):
        """Returns a dataframe with the tracking (position: x, y, z) of all players + the ball
        
        Dropping rows where ball positions is NAN

        +-------+-----------+-----------+
        |       |   BALL    |  TeamId   |
        +       +-----------+-----------+
        |       |  BallId   | PlayerId  |
        + N | T +---+---+---+---+---+---+
        |       | X | Y | Z | X | Y | A |
        +=======+===+===+===+===+===+===+
        | 0 | 0 |123|456|789|012|456|789|
        +-------+-----------+-----------+

        :param xml_positions: the parsed xml tracking data (lxml object)
        :param half: the description of the half: either "firstHalf" or "secondHalf", etc.
        :return: a dataframe containing the tracking data: position of all players and the ball
        """
        def change_type(df, frameset):
            """
            change type of columns
            different columns names and tpyes for the ball and for the rest of the players
            """
            if frameset.attrib.get('TeamId') == 'BALL':
                return df.astype(
                    # change types of columns for the ball
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        'Z':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                        'BallPossession':int, 
                        'BallStatus':int,
                    }, 
                    errors='raise'
                )
            else:
                return df.astype(
                    # change types of columns for the players
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                    }, 
                    errors='raise'
                )
        

        def create_columns(df, frameset):
            """A utility function to create a multiindex columns"""
            person_id = frameset.attrib.get('PersonId')
            team_id = frameset.attrib.get('TeamId')
            df.columns = pd.MultiIndex.from_product(
                [[team_id], [person_id], df.columns],
                names=['TeamId', 'PersonId', 'Position']
            )
            return df

        tracking_players = pd.concat([
            pd.DataFrame.from_records(
                [dict(frame.attrib) for frame in frameset.getchildren()]
            ).pipe(change_type, frameset=frameset)
             .set_index(['N', 'T'])
             .pipe(create_columns, frameset=frameset)
            for frameset in xml_positions.xpath(F'//Positions/FrameSet[@GameSection = "{half}"]')
        ], axis=1, sort=False)
        
        # this is casting the 'T' values in the index to datetime with the correct time zone
        tracking_players.index.set_levels(pd.to_datetime(tracking_players.index.get_level_values(1)), level=1, inplace = True)

        return tracking_players

In [8]:
positions=get_tracking_data(xml_positions)     
##create dataframe from xml input

In [90]:
positions2=get_tracking_data(xml_positions,half='secondHalf')

In [9]:
def calculateDistance(x1,y1,x2,y2):  
     dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
     return dist  

In [11]:
print (calculateDistance(32.4,15.3,48,12.5))

15.849290204927161


In [10]:
CUID1 = 'DFL-CLU-000N8Y'
cuid1_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID1)
cuid1_team_data = DataReader(path+cuid1_player_info)
cuid1_team_meta_df = cuid1_team_data.create_dataframe()
cuid1_player_IDs = cuid1_team_meta_df.ObjectId.tolist()

In [11]:
CUID2 = 'DFL-CLU-000N8Z'
cuid2_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID2)
cuid2_team_data = DataReader(path+cuid2_player_info)
cuid2_team_meta_df = cuid2_team_data.create_dataframe()
cuid2_player_IDs = cuid2_team_meta_df.ObjectId.tolist()

In [19]:
Germany=positions.loc[:,('DFL-CLU-000N8Y',slice(None),['X','Y'])] 

In [20]:
german_players = Germany.columns.get_level_values(level=1).unique()

In [24]:
Germany['DFL-CLU-000N8Y',german_players[0],'DtB']=0

C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
('DFL-OBJ-0000I4') in positions.columns.get_level_values(level=1)

True

In [40]:
i=3
calculateDistance(Germany['DFL-CLU-000N8Y','DFL-OBJ-000191','X'][i],Germany['DFL-CLU-000N8Y','DFL-OBJ-000191','Y'][i],positions['BALL','DFL-OBJ-0000XT','X'][i],positions['BALL','DFL-OBJ-0000XT','Y'][i])

9.950778863988488

In [12]:
players=positions.columns.get_level_values(level=1).unique()

In [93]:
positions[CUID1]['DFL-OBJ-000191']

,Position,X,Y,S,D,A,M
N,T,,,,,,
10000,2019-06-11 20:47:01.600000+02:00,-8.60,-4.81,0.00,0.00,0.00,1
10001,2019-06-11 20:47:01.640000+02:00,-8.59,-4.80,1.25,1.38,-0.05,1
10002,2019-06-11 20:47:01.680000+02:00,-8.59,-4.79,1.25,1.55,0.00,1
10003,2019-06-11 20:47:01.720000+02:00,-8.58,-4.78,1.25,1.55,0.06,1
10004,2019-06-11 20:47:01.760000+02:00,-8.57,-4.77,1.27,1.55,0.13,1
...,...,...,...,...,...,...,...
77670,2019-06-11 21:32:08.400000+02:00,-23.55,-2.93,16.86,18.34,0.00,46
77671,2019-06-11 21:32:08.440000+02:00,-23.37,-2.91,17.00,0.00,0.00,46
77672,2019-06-11 21:32:08.480000+02:00,-23.18,-2.90,17.01,0.00,0.00,46


In [13]:
players=players.drop("DFL-OBJ-0000XT")  #deletes the ball from the list of players

In [100]:
players2=positions2.columns.get_level_values(level=1).unique()

In [114]:
players2=players2.drop("DFL-OBJ-0000XT") #deletes the ball from the list of players

In [80]:
('DFL-OBJ-002FZY') in cuid2_player_IDs

True

In [103]:
half={} # frames der kickoff zeiten in frames
kickoffs=kpimergednew[(kpimergednew['SUBTYPE']=='Kickoff')].index.tolist()
for i in kickoffs:
    if i != 0:
        framedif=kpimergednew.iloc[i].N -kpimergednew.iloc[i-2].N
        if framedif > 10000:
            half['2nd half']=kpimergednew.iloc[i].N
    else:
        half['1st half']=kpimergednew.iloc[i].N

In [104]:
half

{'1st half': 10008, '2nd half': 100046}

In [142]:
i=1
lol=str(i)+ '. Substitution ' + CUID1
lol

'1. Substitution DFL-CLU-000N8Y'

In [16]:
substitutions={}
subs=kpimergednew[kpimergednew.SUBTYPE=='Substitution']
i=1
j=1
for index,rows in subs.iterrows():
    if subs.loc[index,'PUID1'] in cuid1_player_IDs:
        substitutions[str(i)+ '. Substitution ' + CUID1]=subs.loc[index,'PUID1']
        i=i+1
    else:
        substitutions[str(j)+ '. Substitution ' + CUID2]=subs.loc[index,'PUID1']
        j=j+1
        

In [17]:
substitutions

{'1. Substitution DFL-CLU-000N8Y': 'DFL-OBJ-0000SE',
 '2. Substitution DFL-CLU-000N8Y': 'DFL-OBJ-0000OJ',
 '1. Substitution DFL-CLU-000N8Z': 'DFL-OBJ-002G05',
 '3. Substitution DFL-CLU-000N8Y': 'DFL-OBJ-0000O3',
 '2. Substitution DFL-CLU-000N8Z': 'DFL-OBJ-002FZW',
 '3. Substitution DFL-CLU-000N8Z': 'DFL-OBJ-002FZZ'}

In [27]:
list(subs.loc[:,'PUID2'])[0]

'DFL-OBJ-0001F4'

In [14]:
for player in players:
    dtb=[]
    if player in cuid1_player_IDs:
        for index,row in positions.iterrows():
            dtb.append(calculateDistance(positions[CUID1,player,'X'][index],positions[CUID1,player,'Y'][index],positions['BALL','DFL-OBJ-0000XT','X'][index],positions['BALL','DFL-OBJ-0000XT','Y'][index]))
        positions[CUID1][player]['DtB']=dtb  
        #print(len(dtb))
    if player in cuid2_player_IDs: 
        for index,row in positions.iterrows():
            dtb.append(calculateDistance(positions[CUID2,player,'X'][index],positions[CUID2,player,'Y'][index],positions['BALL','DFL-OBJ-0000XT','X'][index],positions['BALL','DFL-OBJ-0000XT','Y'][index]))
        positions[CUID2][player]['DtB']=dtb
        #print(len(dtb))

In [85]:
#calculates distance to ball for every player playing 1.half
for player in players:
    if player in cuid1_player_IDs:
        positions[CUID1,player,'DtB']=positions.apply(lambda x: calculateDistance(x[CUID1,player,'X'],x[CUID1,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)
    if player in cuid2_player_IDs:
        positions[CUID2,player,'DtB']=positions.apply(lambda x: calculateDistance(x[CUID2,player,'X'],x[CUID2,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)

In [116]:
#calculates distance to ball for every player playing 2.half
for player in players:
    if player in cuid1_player_IDs:
        positions2[CUID1,player,'DtB']=positions2.apply(lambda x: calculateDistance(x[CUID1,player,'X'],x[CUID1,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)
    if player in cuid2_player_IDs:
        positions2[CUID2,player,'DtB']=positions2.apply(lambda x: calculateDistance(x[CUID2,player,'X'],x[CUID2,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)

KeyError: (('DFL-CLU-000N8Y', 'DFL-OBJ-0000SE', 'X'), 'occurred at index (100000, 2019-06-11 21:47:19.320000+02:00)')

In [89]:
positions=positions.sort_index(axis=1)

In [118]:
positions2[CUID1]['DFL-OBJ-0000SE']

KeyError: 'DFL-OBJ-0000SE'

In [47]:
players[0] in cuid2_player_IDs

False

In [84]:
dtb=positions.apply(lambda x: calculateDistance(x[CUID1,players[1],'X'],x[CUID1,players[1],'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)
positions[CUID1,players[1],'DtB']=dtb
positions = positions.sort_index(axis=1)
positions[CUID1][players[1]]

,Position,A,D,DtB,M,S,X,Y
N,T,,,,,,,
10000,2019-06-11 20:47:01.600000+02:00,0.00,0.00,0.308058,1,0.00,0.18,0.01
10001,2019-06-11 20:47:01.640000+02:00,0.10,3.32,0.308058,1,0.37,0.18,0.01
10002,2019-06-11 20:47:01.680000+02:00,-0.02,3.42,0.308058,1,0.37,0.18,0.01
10003,2019-06-11 20:47:01.720000+02:00,-0.19,3.42,0.322025,1,0.37,0.19,0.00
10004,2019-06-11 20:47:01.760000+02:00,-0.38,3.42,0.322025,1,0.37,0.19,0.00
10005,2019-06-11 20:47:01.800000+02:00,-0.60,3.42,0.330151,1,0.36,0.19,-0.01
10006,2019-06-11 20:47:01.840000+02:00,-0.82,3.42,0.330151,1,0.31,0.19,-0.01
10007,2019-06-11 20:47:01.880000+02:00,-1.02,3.42,0.338378,1,0.44,0.19,-0.02
10008,2019-06-11 20:47:01.920000+02:00,-0.79,3.42,0.590085,1,0.36,0.18,-0.03


In [78]:
nan_value = float("NaN")
positions[CUID1][players[0]]['X']

N      T                               
10000  2019-06-11 20:47:01.600000+02:00    -8.60
10001  2019-06-11 20:47:01.640000+02:00    -8.59
10002  2019-06-11 20:47:01.680000+02:00    -8.59
10003  2019-06-11 20:47:01.720000+02:00    -8.58
10004  2019-06-11 20:47:01.760000+02:00    -8.57
10005  2019-06-11 20:47:01.800000+02:00    -8.56
10006  2019-06-11 20:47:01.840000+02:00    -8.55
10007  2019-06-11 20:47:01.880000+02:00    -8.54
10008  2019-06-11 20:47:01.920000+02:00    -8.52
10009  2019-06-11 20:47:01.960000+02:00    -8.50
10010  2019-06-11 20:47:02+02:00           -8.49
10011  2019-06-11 20:47:02.040000+02:00    -8.48
10012  2019-06-11 20:47:02.080000+02:00    -8.46
10013  2019-06-11 20:47:02.120000+02:00    -8.45
10014  2019-06-11 20:47:02.160000+02:00    -8.42
10015  2019-06-11 20:47:02.200000+02:00    -8.40
10016  2019-06-11 20:47:02.240000+02:00    -8.39
10017  2019-06-11 20:47:02.280000+02:00    -8.37
10018  2019-06-11 20:47:02.320000+02:00    -8.35
10019  2019-06-11 20:47:02.36

In [77]:
positions[CUID1][players[0]]['DtB']

KeyError: 'DtB'

In [64]:
positions.loc[:,(slice(None),players[0],['DTB'])]=pd.Series(positions.apply(lambda x: calculateDistance(x[CUID1,players[0],'X'],x[CUID1,players[0],'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1))

KeyError: ('Key length (3) exceeds index depth (2)', 'occurred at index (BALL, DFL-OBJ-0000XT, A)')

In [17]:
for player in Germany:
    Germany

TeamId                                 DFL-CLU-000N8Y                       \
PersonId                               DFL-OBJ-000191       DFL-OBJ-0027G6   
Position                                            X     Y              X   
N     T                                                                      
10000 2019-06-11 20:47:01.600000+02:00          -8.60 -4.81           0.18   
10001 2019-06-11 20:47:01.640000+02:00          -8.59 -4.80           0.18   
10002 2019-06-11 20:47:01.680000+02:00          -8.59 -4.79           0.18   
10003 2019-06-11 20:47:01.720000+02:00          -8.58 -4.78           0.19   
10004 2019-06-11 20:47:01.760000+02:00          -8.57 -4.77           0.19   

TeamId                                                             \
PersonId                                     DFL-OBJ-0000I4         
Position                                   Y              X     Y   
N     T                                                             
10000 2019-06-11 20:47:01.600000+02:00  0.01         -47.11 -0.26   
10001 2019-06-11 20:47:01.640000+02:00  0.01         -47.06 -0.23   
10002 2019-06-11 20:47:01.680000+02:00  0.01         -47.01 -0.19   
10003 2019-06-11 20:47:01.720000+02:00  0.00         -46.95 -0.17   
10004 2019-06-11 20:47:01.760000+02:00  0.00         -46.91 -0.14   

TeamId                                                                       \
PersonId                               DFL-OBJ-0000SE        DFL-OBJ-0001HT   
Position                                            X      Y              X   
N     T                                                                       
10000 2019-06-11 20:47:01.600000+02:00         -13.14  16.98         -21.19   
10001 2019-06-11 20:47:01.640000+02:00         -13.13  16.96         -21.19   
10002 2019-06-11 20:47:01.680000+02:00         -13.13  16.92         -21.19   
10003 2019-06-11 20:47:01.720000+02:00         -13.12  16.90         -21.19   
10004 2019-06-11 20:47:01.760000+02:00         -13.12  16.86         -21.19   

TeamId                                        ...                       \
PersonId                                      ... DFL-OBJ-0000OJ         
Position                                   Y  ...              X     Y   
N     T                                       ...                        
10000 2019-06-11 20:47:01.600000+02:00  8.91  ...          -9.31  4.89   
10001 2019-06-11 20:47:01.640000+02:00  8.91  ...          -9.30  4.91   
10002 2019-06-11 20:47:01.680000+02:00  8.91  ...          -9.30  4.93   
10003 2019-06-11 20:47:01.720000+02:00  8.91  ...          -9.30  4.95   
10004 2019-06-11 20:47:01.760000+02:00  8.91  ...          -9.31  4.96   

TeamId                                                                       \
PersonId                               DFL-OBJ-0000ZS        DFL-OBJ-0002F5   
Position                                            X      Y              X   
N     T                                                                       
10000 2019-06-11 20:47:01.600000+02:00         -18.07 -10.54          -9.97   
10001 2019-06-11 20:47:01.640000+02:00         -18.07 -10.57          -9.97   
10002 2019-06-11 20:47:01.680000+02:00         -18.08 -10.59          -9.97   
10003 2019-06-11 20:47:01.720000+02:00         -18.08 -10.61          -9.97   
10004 2019-06-11 20:47:01.760000+02:00         -18.09 -10.62          -9.98   

TeamId                                                              \
PersonId                                     DFL-OBJ-0000O3          
Position                                   Y              X      Y   
N     T                                                              
10000 2019-06-11 20:47:01.600000+02:00  0.62          -0.01  11.47   
10001 2019-06-11 20:47:01.640000+02:00  0.62           0.02  11.47   
10002 2019-06-11 20:47:01.680000+02:00  0.63           0.07  11.47   
10003 2019-06-11 20:47:01.720000+02:00  0.63           0.11  11.48   
10004 2019-06-11 20:47:0

In [21]:
substitutions['3. Substitution DFL-CLU-000N8Y'] in positions.columns.get_level_values(level=1).unique()

True

In [30]:
list(subs.loc[:,'PUID2'])[0]in players

False

In [41]:
i=1
print(subs.iloc[i,:].PUID1)
print(subs.iloc[i,:].PUID2)
print(subs.iloc[i,:].N)

DFL-OBJ-0000OJ
DFL-OBJ-0000Q9
110968


In [42]:
player_id='DFL-OBJ-0000OJ'
positions.loc[
    # index (rows)
    (
        slice(110960,110970),
    ),
    # columns
    (
        slice(None), # team level
        player_id,   # player level
        ['X','Y']    # position level
    )
]

Empty DataFrame
Columns: [(DFL-CLU-000N8Y, DFL-OBJ-0000OJ, X), (DFL-CLU-000N8Y, DFL-OBJ-0000OJ, Y)]
Index: []

In [48]:
positions2.columns.get_level_values(level=1).unique()

Index(['DFL-OBJ-0000XT', 'DFL-OBJ-0027G6', 'DFL-OBJ-000191', 'DFL-OBJ-0001F4',
       'DFL-OBJ-002G0E', 'DFL-OBJ-002FZX', 'DFL-OBJ-002G01', 'DFL-OBJ-0000I4',
       'DFL-OBJ-002FZQ', 'DFL-OBJ-002FZP', 'DFL-OBJ-002G03', 'DFL-OBJ-002G08',
       'DFL-OBJ-0001HT', 'DFL-OBJ-0002AU', 'DFL-OBJ-0000OJ', 'DFL-OBJ-0000ZS',
       'DFL-OBJ-0002F5', 'DFL-OBJ-002FZY', 'DFL-OBJ-0029Y8', 'DFL-OBJ-002G05',
       'DFL-OBJ-002FZR', 'DFL-OBJ-0000O3', 'DFL-OBJ-002FZW', 'DFL-OBJ-00021U',
       'DFL-OBJ-00258K', 'DFL-OBJ-002FZZ', 'DFL-OBJ-0000Q9', 'DFL-OBJ-002G0B'],
      dtype='object', name='PersonId')

In [21]:
positions.columns.get_level_values(level=2).unique()

Index(['A', 'BallPossession', 'BallStatus', 'D', 'M', 'S', 'X', 'Y', 'Z'], dtype='object', name='Position')

In [91]:
positions2

TeamId                                            BALL                 \
PersonId                                DFL-OBJ-0000XT                  
Position                                             A BallPossession   
N      T                                                                
100000 2019-06-11 21:47:19.320000+02:00            0.0              2   
100001 2019-06-11 21:47:19.360000+02:00            0.0              2   
100002 2019-06-11 21:47:19.400000+02:00            0.0              2   
100003 2019-06-11 21:47:19.440000+02:00            0.0              2   
100004 2019-06-11 21:47:19.480000+02:00            0.0              2   
100005 2019-06-11 21:47:19.520000+02:00            0.0              2   
100006 2019-06-11 21:47:19.560000+02:00            0.0              2   
100007 2019-06-11 21:47:19.600000+02:00            0.0              2   
100008 2019-06-11 21:47:19.640000+02:00            0.0              2   
100009 2019-06-11 21:47:19.680000+02:00            0.0              2   
100010 2019-06-11 21:47:19.720000+02:00            0.0              2   
100011 2019-06-11 21:47:19.760000+02:00            0.0              2   
100012 2019-06-11 21:47:19.800000+02:00            0.0              2   
100013 2019-06-11 21:47:19.840000+02:00            0.0              2   
100014 2019-06-11 21:47:19.880000+02:00            0.0              2   
100015 2019-06-11 21:47:19.920000+02:00            0.0              2   
100016 2019-06-11 21:47:19.960000+02:00            0.0              2   
100017 2019-06-11 21:47:20+02:00                   0.0              2   
100018 2019-06-11 21:47:20.040000+02:00            0.0              2   
100019 2019-06-11 21:47:20.080000+02:00            0.0              2   
100020 2019-06-11 21:47:20.120000+02:00            0.0              2   
100021 2019-06-11 21:47:20.160000+02:00            0.0              2   
100022 2019-06-11 21:47:20.200000+02:00            0.0              2   
100023 2019-06-11 21:47:20.240000+02:00            0.0              2   
100024 2019-06-11 21:47:20.280000+02:00            0.0              2   
100025 2019-06-11 21:47:20.320000+02:00            0.0              2   
100026 2019-06-11 21:47:20.360000+02:00            0.0              2   
100027 2019-06-11 21:47:20.400000+02:00            0.0              2   
100028 2019-06-11 21:47:20.440000+02:00            0.0              2   
100029 2019-06-11 21:47:20.480000+02:00            0.0              2   
...                                                ...            ...   
170470 2019-06-11 22:34:18.120000+02:00            0.0              2   
170471 2019-06-11 22:34:18.160000+02:00            0.0              2   
170472 2019-06-11 22:34:18.200000+02:00            0.0              2   
170473 2019-06-11 22:34:18.240000+02:00            0.0              2   
170474 2019-06-11 22:34:18.280000+02:00            0.0              2   
170475 2019-06-11 22:34:18.320000+02:00            0.0              2   
170476 2019-06-11 22:34:18.360000+02:00            0.0              2   
170477 2019-06-11 22:34:18.400000+02:00            0.0              2   
170478 2019-06-11 22:34:18.440000+02:00            0.0              2   
170479 2019-06-11 22:34:18.480000+02:00            0.0              2   
170480 2019-06-11 22:34:18.520000+02:00            0.0              2   
170481 2019-06-11 22:34:18.560000+02:00            0.0              2   
170482 2019-06-11 22:34:18.600000+02:00            0.0              2   
170483 2019-06-11 22:34:18.640000+02:00            0.0              2   
170484 2019-06-11 22:34:18.680000+02:00            0.0              2   
170485 2019-06-11 22:34:18.720000+02:00            0.0              2   
170486 2019-06-11 22:34:18.760000+02:00            0.0              2   
170487 2019-06-11 22:34:18.800000+02:00            0.0              2   
170488 2019-06-11 22:34:18.840000+02:00            0.0              2   
170489 2019-06-11 22:34:18.880000+02:00            0.0    